In [1]:
from dotenv import load_dotenv
import os
load_dotenv()

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY")
os.environ["LANGSMITH_PROJECT"] = "ollama simple app"
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [2]:
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser
llm = ChatGroq(model="llama-3.3-70b-versatile")

In [3]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "Translate this text into {language} you do not have to specify any kind of information in english just translate "),
    ("human", "{input}")
])

In [4]:
parser = StrOutputParser()

In [5]:
chain = prompt | llm | parser

In [6]:
res = chain.invoke({"language":"japanese", "input":"hello my name is anuj"})
print(res)

こんにちは、私の名前はアヌジュです


## simple GenAI app that can remember with sessions

## Building A Chatbot
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [7]:
from langchain_core.messages import HumanMessage
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables import RunnableWithMessageHistory

store = {} # this stores all the session id

def get_session_history(session_id:str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [8]:
with_message_history = RunnableWithMessageHistory(llm,get_session_history)

In [9]:
config = {"configurable":{"session_id":"chat1"}}

In [10]:
with_message_history.invoke(
    [HumanMessage(content="Hi, I am anuj. i am currently learning gen AI"),
     ],config=config
)

AIMessage(content="Hello Anuj, nice to meet you. That's great to hear that you're learning about General AI (Gen AI). It's an exciting and rapidly evolving field that has the potential to transform many aspects of our lives.\n\nWhat specific areas of Gen AI are you interested in or currently learning about? Are you exploring topics like natural language processing, computer vision, reinforcement learning, or something else?\n\nAlso, what's your background and what motivated you to learn about Gen AI? Are you looking to apply your knowledge in a particular industry or domain? I'm here to help and provide any guidance or resources I can.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 124, 'prompt_tokens': 48, 'total_tokens': 172, 'completion_time': 0.333705232, 'prompt_time': 0.00237133, 'queue_time': 0.056452020000000006, 'total_time': 0.336076562}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_9a8b91ba77', 'service_tier': 'on_demand

In [11]:
with_message_history.invoke(
    [HumanMessage(content="what is my name? and do you know what i am learning right now?"),
     ],config=config
)

AIMessage(content='Your name is Anuj, and according to our conversation, you are currently learning about General AI (Gen AI).', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 197, 'total_tokens': 221, 'completion_time': 0.083806183, 'prompt_time': 0.009890477, 'queue_time': 0.052817353000000004, 'total_time': 0.09369666}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_3f3b593e33', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--513a1db7-73ef-4572-aeec-bc4c198baeb3-0', usage_metadata={'input_tokens': 197, 'output_tokens': 24, 'total_tokens': 221})

In [12]:
config_2 = {"configurable":{"session_id":"chat2"}}
with_message_history.invoke(
    [HumanMessage(content="Hi, do you know my name?"),
     ],config=config_2
)

AIMessage(content="I don't have any information about your name. I'm a large language model, I don't have the ability to recall previous conversations or maintain personal information about users. Each time you interact with me, it's a new conversation. If you'd like to share your name, I'd be happy to chat with you and address you by name!", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 71, 'prompt_tokens': 43, 'total_tokens': 114, 'completion_time': 0.141565267, 'prompt_time': 0.002615213, 'queue_time': 0.052159057, 'total_time': 0.14418048}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_6507bcfb6f', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--5cff803f-8c09-4929-a3b9-c36c34320eaf-0', usage_metadata={'input_tokens': 43, 'output_tokens': 71, 'total_tokens': 114})

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [13]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Amnswer all the question to the nest of your ability, I want you to answer in this {language}.",),
        MessagesPlaceholder(variable_name="message")
    ]
)
chain = prompt | llm


In [14]:
with_message_history = RunnableWithMessageHistory(chain,get_session_history,input_messages_key="message")

In [15]:
config3 = {"configurable":{"session_id":"chat_6"}}

In [16]:
res = with_message_history.invoke({
    "message":[HumanMessage(content="Hi i am anuj.")],"language":"japanese"
},
config=config3
)
res.content

"(konnichiwa) Anuj-san, watashi wa anata no shitsumon ni kotaete ikimasu. Nihongo de hanasou to omoimasu ka? (Hello Anuj, I'll do my best to answer your questions. Shall we converse in Japanese?)"

In [17]:
res = with_message_history.invoke({
    "message":[HumanMessage(content="what is my name")],"language":"japanese"
},
config=config3
)
res.content

"(anata no namae wa) Anuj desu ne. (Your name is Anuj, isn't it?)"

## Managing the conversation history

In [20]:
from langchain_core.messages import SystemMessage,trim_messages, AIMessage
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=llm,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

The chain is set up to assign messages using itemgetter, retrieve them from the prompt template, and apply the trimmer. The chain is then concatenated with the prompt and the model.

In [ ]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(message = itemgetter("message")| trimmer)
    | prompt
    | llm
)

In [26]:
response = chain.invoke(
    {
        "message": messages + [HumanMessage(content="what ice cream did i like ?")],
        "language": "English"
    }
)
response.content

"I don't know, you didn't tell me! This is the start of our conversation, I don't have any information about your favorite ice cream. Want to share?"

In [27]:
response = chain.invoke(
    {
        "message": messages + [HumanMessage(content="what math problem did i ask for ?")],
        "language": "English"
    }
)
response.content

'You asked for the answer to 2 + 2.'

## Wraping it all in a message history

In [28]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="message"
)
config = {"configurable":{"session_id":"chat_6"}}

In [32]:
response = with_message_history.invoke(
    {
        "message":messages + [HumanMessage(content="do you know what math problem did i ask?")],
        "language":"english"
    },
    config
)
response.content

'You asked what 2 + 2 is.'

In [33]:
response = with_message_history.invoke(
    {
        "message":messages + [HumanMessage(content="tell me about our conversations")],
        "language":"english"
    },
    config
)
response.content

'We\'ve had a few conversations so far. Here\'s a summary:\n\n* You mentioned that you like vanilla ice cream.\n* You asked me a math question, "what\'s 2 + 2", and I replied that the answer is 4.\n* You thanked me for answering your question.\n* You asked if I was having fun, and I said yes.\n* Now, you\'re asking me to tell you about our conversations, which I\'m doing!\n\nIt\'s been a nice and simple conversation so far! Is there anything else you\'d like to talk about?'